In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Flatten, Dense, Input, Convolution2D, MaxPooling2D
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])

def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 16
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [2]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [3]:
np.shape(Y_train_label)

(4096, 256)

In [4]:
# #Output dim for your dataset
# output_dim = 256 #For Caltech256

# tl_model = getModel( output_dim )
# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
# for epoch in range(10):
#     start = 0
#     for batch in range(len(X_train)/64):
#         coll=io.ImageCollection(X_train[start:start+64])
#         images = convert_image(coll)
#         tl_model.train_on_batch(images, Y_train_label[start:start+64, :])
# #         tl_model.summary()
#         start += 64
# print 'stop here'

In [5]:
#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [6]:
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    layer_name = 'block5_conv3'
    x = vgg_model.get_layer(layer_name).output
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='pool_new')(x)
    x = Flatten(name='flatten')(x)
#     x = Dense(2048, activation='relu', name='fc1_new')(x)
#     x = Dense(4096, activation='relu', name='fc2_new')(x)
    softmax_layer = Dense(256, activation='softmax')(x) #Create softmax layer taking input as vgg_out
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False

    return tl_model

In [7]:
#Model
#Output dim for your dataset
output_dim = 256 #For Caltech256
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error
# sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True)
tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [8]:
print tl_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [9]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [10]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 2048 samples, validate on 2048 samples
Epoch 1/20
2048/2048 [==============================] - 40s - loss: 14.8057 - acc: 0.0386 - val_loss: 14.1845 - val_acc: 0.0630
Epoch 2/20
2048/2048 [==============================] - 41s - loss: 12.6794 - acc: 0.1909 - val_loss: 13.7882 - val_acc: 0.0928
Epoch 3/20
2048/2048 [==============================] - 42s - loss: 11.7421 - acc: 0.2607 - val_loss: 13.4397 - val_acc: 0.1094
Epoch 4/20
2048/2048 [==============================] - 42s - loss: 11.1216 - acc: 0.3022 - val_loss: 13.3828 - val_acc: 0.1094
Epoch 5/20
2048/2048 [==============================] - 43s - loss: 10.7051 - acc: 0.3276 - val_loss: 13.3165 - val_acc: 0.1055
Epoch 6/20
2048/2048 [==============================] - 42s - loss: 10.3596 - acc: 0.3525 - val_loss: 13.4339 - val_acc: 0.1074
Epoch 7/20
2048/2048 [==============================] - 42s - loss: 10.1223 - acc: 0.3677 - val_loss: 13.3691 - val_acc: 0.1094
Epoch 8/20
2048/2048 [==============================] - 

In [11]:
print(hist.history)

{'acc': [0.03857421875, 0.19091796875, 0.2607421875, 0.30224609375, 0.32763671875, 0.3525390625, 0.36767578125, 0.380859375, 0.38818359375, 0.4013671875, 0.40966796875, 0.4169921875, 0.4267578125, 0.4375, 0.44384765625, 0.451171875, 0.45751953125, 0.45849609375, 0.45849609375, 0.45849609375], 'loss': [14.805689841508865, 12.679409220814705, 11.74211211502552, 11.121609650552273, 10.705052986741066, 10.359575062990189, 10.122346840798855, 9.9246909320354462, 9.7908512800931931, 9.6029782369732857, 9.4701602905988693, 9.3444276079535484, 9.1883098930120468, 9.0304140001535416, 8.9443164691329002, 8.7961786538362503, 8.7370554208755493, 8.7280192226171494, 8.7280116826295853, 8.7280116826295853], 'val_acc': [0.06298828125, 0.0927734375, 0.109375, 0.109375, 0.10546875, 0.107421875, 0.109375, 0.11083984375, 0.11962890625, 0.11865234375, 0.12255859375, 0.12158203125, 0.12353515625, 0.1337890625, 0.1279296875, 0.1328125, 0.1318359375, 0.13330078125, 0.13330078125, 0.13330078125], 'val_loss': 

In [9]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto'),
    ModelCheckpoint(filepath="./weights.hdf5", verbose=1, save_best_only=True)
]

In [31]:
# tl_model.fit(X_train_data, Y_train_label,
#           nb_epoch=20,
#           batch_size=16)

tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=1, 
             verbose=1,  
             shuffle=None, class_weight=None, sample_weight=None)

Epoch 1/1
4096/4096 [==============================] - 40s - loss: 0.2756 - acc: 0.9353    


In [33]:
score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=32, verbose=1, sample_weight=None)
print score

2695/2695 [==============================] - 26s    
[1.7824576142546418, 0.59962894253031884]


In [ ]:
import sys

# manual batch
nb_epoch = 20
nm_per_batch = 32
for e in range(nb_epoch):
    print 'Epoch', e
    batches = 0
    for X_batch, Y_batch in datagen.flow(X_train_data, Y_train_label, batch_size=nm_per_batch, shuffle=True):
        loss = tl_model.train_on_batch(X_batch, Y_batch)
#         print 'batches:', batches, ', loss:' , loss[0] , ', acc train:' , loss[1]
#         print 'batches:', batches, 'loss:' , loss[0],
        print 'batches:', batches,
        
        batches += 1
        if batches >= len(X_train) / nm_per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    
    #evaluate on validation
    score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=nm_per_batch, verbose=1, sample_weight=None)
    print 'loss:' , loss[0] , ', acc train:' , loss[1], 'loss train: ', score